In [1]:
## Parameters specific to where your folders are and your data
parameter_file = '../parameters/params.yaml'
import yaml
import sys
import os
with open(parameter_file,'r') as f:
    doc = yaml.load(f)

#p = dic2obj(**doc)

data_folder = doc['data_folder']
tissues = doc['tissues'].split(',')
sys.path.append(doc['pipeline_path'])
ref_fa = doc['ref_fa']
annotation=doc['annotation']

os.chdir(doc["results"])

/home/isshamie/software/anaconda2/envs/TSS/lib/python3.7/site-packages/ipykernel_launcher.py:7: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  import sys


In [2]:
tss_annotation = "genome/picr/gcf/start_site_mRNA.tsv" #doc['tss_annotation']

In [3]:
from collections import defaultdict
import os
import sys
import pandas as pd
import matplotlib
import seaborn as sns
import pickle
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from itertools import product
import glob
import re
from matplotlib_venn import venn2
from matplotlib import rcParams
import inspect
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sys.setrecursionlimit(3000)
%load_ext autoreload
%autoreload 2
rcParams['figure.figsize'] = 8, 6
import tqdm
### notebook specific configuration ###
from os.path import basename
#mpl.style.use('ggplot')
mpl.style.use('fivethirtyeight')
from cycler import cycler
mpl.rcParams['axes.prop_cycle'] = cycler(color='bgrcmyk')

from Homer import *


print('Number of tissues: ',len(tissues))

Number of tissues:  13


In [4]:
meta_samples = pd.read_csv("../parameters/samples.tsv", sep="\t",index_col=0) #pd.read_csv('../raw_metaSamples.tsv',sep="\t",index_col=0)
meta_samples.head()

,Experiment,Full,Name,Tissue
CHBMDM1hKLA2_ATAC_JHS1000_SD_AGGTTGGG_S11_L001_R1_001.fastq.gz,ATAC,CHBMDM1hKLA2_ATAC_JHS1000_SD_AGGTTGGG_S11_L001...,BMDM1hKLA_ATAC1,BMDM1hKLA
CHBMDM1hKLA1_ATAC_JHS998_SD_TGGGTTTC_S9_L001_R1_001.fastq.gz,ATAC,CHBMDM1hKLA1_ATAC_JHS998_SD_TGGGTTTC_S9_L001_R...,BMDM1hKLA_ATAC2,BMDM1hKLA
CHBMDM1hKLA_GRO_JHS1027_SD_CAAAAG_S42_L003_R1_001.fastq.gz,GRO,CHBMDM1hKLA_GRO_JHS1027_SD_CAAAAG_S42_L003_R1_...,BMDM1hKLA_GRO1,BMDM1hKLA
CHBMDM1hKLA_GRO_JHS1034_SD_AGTCAA_S49_L003_R1_001.fastq.gz,GROCap,CHBMDM1hKLA_GRO_JHS1034_SD_AGTCAA_S49_L003_R1_...,BMDM1hKLA_GROCap1,BMDM1hKLA
CHBMDMVehic2_ATAC_JHS999_SD_TGGTCACA_S10_L001_R1_001.fastq.gz,ATAC,CHBMDMVehic2_ATAC_JHS999_SD_TGGTCACA_S10_L001_...,BMDMwt_ATAC1,BMDMwt


In [5]:
def merge_columns(df, mapping_dict):
    '''Function that merges columns by taking the mean of them.
    Merges based on if they have the same element in the meta_samples meta_column.
    Returns:
        new_df: Dataframe but with the columns of interest merged. Also column names are now based
                on the unique meta_samples[meta_column].
    '''

    vals = mapping_dict.keys()#np.unique(meta_samples[meta_column].values)
    new_df = pd.DataFrame(index=df.index, columns=vals)
    for i in vals:
        if not mapping_dict[i] == []:#(meta_samples[meta_column] == i).any():
            new_col = (df.loc[:, mapping_dict[i]])
            new_col = new_col.mean(axis=1)
            new_df.loc[:, i] = new_col
    new_df = new_df.loc[:, ~(new_df.isnull().all())]
    return new_df


In [6]:
peaks_tissue = pd.read_csv("merged/samples.merge.peaksexpression.log10",index_col=0,sep="\t")
peaks_tissue.head()

,Results/f04_peaks/MiscOrgans_START1_and_MiscOrgans_startinput1,Results/f04_peaks/Liver_START2_and_Liver_startinput1,Results/f04_peaks/Brain_GROCap1_and_Brain_GRO1,Results/f04_peaks/Liver_START1_and_Liver_startinput1,Results/f04_peaks/Lung_GROCap1_and_Lung_GRO1,Results/f04_peaks/Brain_START3_and_Brain_startinput1,Results/f04_peaks/Heart_START1_and_Heart_startinput1,Results/f04_peaks/CHO_GROCap2_and_CHO_GRO1,Results/f04_peaks/Heart_START2_and_Heart_startinput1,Results/f04_peaks/Liver_GROCap1_and_Liver_GRO1,...,Results/f04_peaks/MiscOrgans_START2_and_MiscOrgans_startinput1,Results/f04_peaks/Spleen_START1_and_Spleen_startinput1,Results/f04_peaks/Lung_START2_and_Lung_startinput1,Results/f04_peaks/CHO_START1_and_CHO_startinput1,Results/f04_peaks/FemaleReproductive_START1_and_FemaleReproductive_startinput1,Results/f04_peaks/Kidney_START2_and_Kidney_startinput1,Results/f04_peaks/Kidney_GROCap1_and_Kidney_GRO1,Results/f04_peaks/CHO_GROCap1_and_CHO_GRO1,Results/f04_peaks/BMDMwt_GROCap1_and_BMDMwt_GRO1,Results/f04_peaks/Spleen_START2_and_Spleen_startinput1
0,0.382017,0.0,0.356026,0.000000,0.877371,0.000000,0.426511,1.515874,0.0,0.000000,...,0.000000,0.000000,0.000000,0.804821,0.000000,0.0,0.868644,1.531479,0.000000,0.000000
1,0.472756,0.0,0.485721,0.633468,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.843855,0.311754,1.619093,0.000000,0.825426,0.0,0.000000,0.000000,0.000000,0.827369
2,0.382017,0.0,0.290480,0.000000,0.680336,0.000000,0.000000,1.372912,0.0,0.000000,...,0.000000,0.000000,0.000000,0.893207,0.436163,0.0,0.451786,1.354108,0.823474,0.545307
3,0.694605,0.0,0.000000,0.000000,0.000000,0.627366,0.426511,0.000000,0.0,0.000000,...,0.491362,0.649335,0.000000,0.617000,0.000000,0.0,0.000000,0.559907,0.537819,0.517196
4,2.220892,0.0,0.456366,1.918030,0.000000,1.648360,1.717671,0.000000,0.0,0.919078,...,1.776701,1.283301,0.000000,1.968016,0.000000,0.0,0.401401,0.761928,0.642465,2.203848


In [7]:
tissue_to_peaks = defaultdict(list)
for val in peaks_tissue.columns.values:
    count = 0
    for t in np.unique(meta_samples["Tissue"].values):
        if t in val:
            count += 1
            tissue_to_peaks[t].append(val)
            if count > 1:
                print("This sample has multiple tissues associated with it")
                print("Sample: ",val)

In [8]:
peaks_tissue_merged = merge_columns(peaks_tissue,mapping_dict=tissue_to_peaks)

In [9]:
peaks_tissue_merged.head()

,MiscOrgans,Liver,Brain,Lung,Heart,CHO,Pancreas,BMDM1hKLA,FemaleReproductive,Muscle,Kidney,Spleen,BMDMwt
0,0.191009,0.000000,0.192445,0.292457,0.213256,1.379905,0.000000,1.93044,0.177054,0.192803,0.289548,0.000000,0.000000
1,0.658306,0.211156,0.365353,0.875756,0.000000,0.000000,0.000000,0.00000,0.704879,0.160073,0.206016,0.569562,0.000000
2,0.191009,0.000000,0.285237,0.440272,0.000000,1.269450,0.299942,0.00000,0.650337,0.252575,0.377071,0.272654,0.823474
3,0.592983,0.000000,0.633209,0.256951,0.213256,0.347363,0.380588,0.00000,0.556972,0.300486,0.216445,0.583265,0.537819
4,1.998797,0.945703,1.707621,0.729080,0.858835,0.698374,1.141651,0.00000,1.218955,1.198361,0.838893,1.743575,0.642465


In [10]:
peaks_tissue_merged.to_csv("merged/tissues.merge.peaksexpression.log10",sep="\t")

## Same but without log

In [11]:
peaks_tissue = pd.read_csv("merged/samples.merge.peaksexpression",index_col=0,sep="\t")
peaks_tissue.head()

tissue_to_peaks = defaultdict(list)
for val in peaks_tissue.columns.values:
    count = 0
    for t in np.unique(meta_samples["Tissue"].values):
        if t in val:
            count += 1
            tissue_to_peaks[t].append(val)
            if count > 1:
                print("This sample has multiple tissues associated with it")
                print("Sample: ",val)

peaks_tissue_merged = merge_columns(peaks_tissue,mapping_dict=tissue_to_peaks)

peaks_tissue_merged.head()

peaks_tissue_merged.to_csv("merged/tissues.merge.peaksexpression",sep="\t")

## Do the same but for txn_df and gene_df

In [12]:
txn_df = pickle.load(open("tss_annotation/picr/gcf/txn_df_02.p","rb"))

In [13]:
## invert and create a peaks_to_tissue
peaks_to_tissue = dict()
for t in tissue_to_peaks:
    for p in tissue_to_peaks[t]:
        peaks_to_tissue[os.path.basename(p)] = t

In [14]:
txn_df["Tissues"] = ""
for ind,val in tqdm.tqdm(txn_df.iterrows()):
    if len(val["samples"]) != 0:
        tis = set()
        for t in val["samples"]:
            tis.add(peaks_to_tissue[t])
        txn_df.at[ind,"Tissues"] = ",".join(tis)

35679it [00:24, 1465.85it/s]


In [15]:
pickle.dump(txn_df,open("tss_annotation/picr/gcf/txn_df_02_tissues.p","wb"))


In [16]:
gene_df = pickle.load(open("tss_annotation/picr/gcf/gene_df_02.p","rb"))

## invert and create a peaks_to_tissue
peaks_to_tissue = dict()
for t in tissue_to_peaks:
    for p in tissue_to_peaks[t]:
        peaks_to_tissue[os.path.basename(p)] = t

gene_df["Tissues"] = ""
for ind,val in tqdm.tqdm(gene_df.iterrows()):
    if len(val["samples"]) != 0:
        tis = set()
        for t in val["samples"]:
            tis.add(peaks_to_tissue[t])
        gene_df.at[ind,"Tissues"] = ",".join(tis)

pickle.dump(gene_df,open("tss_annotation/gene_df_02_tissues.p","wb"))


21387it [00:08, 2384.18it/s]
